In [1]:
import numpy
import pandas 
import csv
import re
import nltk


#Análisis de sentimientos
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [2]:
%run "ClasificadorFunciones.ipynb"

9199
[0.00010871 0.00021741 0.00054354 ... 0.00054354 0.00010871 0.00010871]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2037 entries, 0 to 2036
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Palabra  2037 non-null   object
 1   PFA      2037 non-null   object
dtypes: object(2)
memory usage: 32.0+ KB


In [3]:
dataTweets = pandas.read_csv(".//TweetsTest//FinalUTF8.txt", names=['Tweets'])
dataTweets = dataTweets.dropna()
dataTweets.info()
dataTweets

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392 entries, 0 to 1391
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweets  1392 non-null   object
dtypes: object(1)
memory usage: 21.8+ KB


,Tweets
0,https://t.co/6cmSHgMXJo
1,RT @AkemiLook: UPDATE: I just got off a call w...
2,RT @AkemiLook: CALLING LA LAWYERS: LA is in de...
3,"Pablo Guerrero Cañez, luchador social de Mexic..."
4,Sounds like Wolvie will be getting a cool new ...
...,...
1387,¿Cuántos juegos han comprado para su smartphon...
1388,#maskism 😷\n_______________________________\nn...
1389,🎉 ¡EL NOMBRE DE LA ROSA de Umberto Eco en la #...
1390,Conchas rellenas de delicioso gelato. ¿Tenemos...


In [29]:
listTweets = dataTweets.Tweets.values.tolist()
print(len(listTweets))
cleanTweets =[]
pattern = re.compile('https?://(?:[-\w./]|(?:%\da-fA-F{2}))+')

1392


In [30]:
for tweet in listTweets:
    res = re.findall(pattern, tweet)
    if (len(res) != 0):
        aux = tweet.replace(res[0], '')
        cleanTweets.append(aux)
    else:
        cleanTweets.append(tweet)
        


In [34]:
#print(cleanTweets)
tweets = numpy.asarray(cleanTweets)

In [35]:
Covid = []
Tecnologia = []
Deportes = []
Economia =[]
Cultura = []

for tweet in tweets:
    result = MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep)
    if result == 'covid':
        Covid.append([tweet, result])
    if result == 'tecnologia':
        Tecnologia.append([tweet, result])
    if result == 'deportes':
        Deportes.append([tweet, result])
    if result == 'economia':
        Economia.append([tweet, result])
    if result == 'cultura':
        Cultura.append([tweet, result])


In [36]:
print("Covid = ", len(Covid))
print("Tecnología = ", len(Tecnologia))
print("Deportes = ", len(Deportes))
print("Economía = ", len(Economia))
print("Cultura = ", len(Cultura))

Covid =  148
Tecnología =  1215
Deportes =  0
Economía =  3
Cultura =  26


In [37]:
cov = numpy.asarray(Covid)
tec = numpy.asarray(Tecnologia)
dep = numpy.asarray(Deportes)
eco = numpy.asarray(Economia)
cul = numpy.asarray(Cultura)

## Se evaluara mediante el análisis de sentimientos cada tópico

In [38]:
Pos = 0
Neg = 0
Neu = 0


for i in range(len(cov)):
    tweet = Covid[i][0]
    vs = SIA.polarity_scores(tweet)
    #print(vs)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #print(auxArr)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    #print(values)
    compound = values[3]
    #print(compound)

    if(compound >= 0.05):
        Pos += 1
    if(compound <= -0.05):
        Neg += 1
    if (compound > -0.05 and compound < 0.05):
        Neu += 1

        
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  0 Negativos =  1 Neutros =  147


In [39]:
Pos = 0
Neg = 0
Neu = 0


for i in range(len(tec)):
    tweet = Tecnologia[i][0]
    vs = SIA.polarity_scores(tweet)
    #print(vs)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #print(auxArr)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    #print(values)
    compound = values[3]
    #print(compound)

    if(compound >= 0.05):
        Pos += 1
    if(compound <= -0.05):
        Neg += 1
    if (compound > -0.05 and compound < 0.05):
        Neu += 1

        
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  272 Negativos =  159 Neutros =  784


In [40]:
Pos = 0
Neg = 0
Neu = 0


for i in range(len(dep)):
    tweet = Deportes[i][0]
    vs = SIA.polarity_scores(tweet)
    #print(vs)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #print(auxArr)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    #print(values)
    compound = values[3]
    #print(compound)

    if(compound >= 0.05):
        Pos += 1
    if(compound <= -0.05):
        Neg += 1
    if (compound > -0.05 and compound < 0.05):
        Neu += 1

        
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  0 Negativos =  0 Neutros =  0


In [41]:
Pos = 0
Neg = 0
Neu = 0


for i in range(len(eco)):
    tweet = Economia[i][0]
    vs = SIA.polarity_scores(tweet)
    #print(vs)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #print(auxArr)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    #print(values)
    compound = values[3]
    #print(compound)

    if(compound >= 0.05):
        Pos += 1
    if(compound <= -0.05):
        Neg += 1
    if (compound > -0.05 and compound < 0.05):
        Neu += 1

        
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  1 Negativos =  0 Neutros =  2


In [42]:
Pos = 0
Neg = 0
Neu = 0


for i in range(len(cul)):
    tweet = Cultura[i][0]
    vs = SIA.polarity_scores(tweet)
    #print(vs)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #print(auxArr)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    #print(values)
    compound = values[3]
    #print(compound)

    if(compound >= 0.05):
        Pos += 1
    if(compound <= -0.05):
        Neg += 1
    if (compound > -0.05 and compound < 0.05):
        Neu += 1

        
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  8 Negativos =  1 Neutros =  17
